# FDO Connect

## FDO Nanopub Operations

This notebook provides examples for the operations that can be performed on FDOs as part of FDO Connect project to implement FAIR Digital Objects (FDOs) based on the existing Handle-based FDO One infrastructure and the technology of nanopublications.

In [12]:
import rdflib
from rdflib.namespace import RDF, RDFS, SH, XSD
from nanopub import (
    definitions,
    Nanopub,
    NanopubClient,
    NanopubRetract,
    load_profile,
    NanopubConf,
)
from nanopub.fdo import (
    FdoNanopub,
    FdoRecord,
    update_record,
    retrieve_record_from_id,
    retrieve_content_from_id,
    validate_fdo_record,
    resolve_id
)
import datetime
from nanopub.namespaces import HDL, FDOF, PROV, NPX, NP
from pyshacl import validate
from rdflib import Graph, URIRef, Literal
import requests
import json


The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [13]:
## Setup

conf = NanopubConf(
    add_prov_generated_time=False,
    add_pubinfo_generated_time=True,
    attribute_assertion_to_profile=True,
    attribute_publication_to_profile=True,
    profile=load_profile(),
    use_test_server=True
)

## Define identifiers
iri = "https://w3id.org/np/RAsSeIyT03LnZt3QvtwUqIHSCJHWW1YeLkyu66Lg4FeBk/nanodash-readme"
handle_iri = "https://hdl.handle.net/21.T11966/82045bd97a0acce88378"
handle_pid = "21.T11966/82045bd97a0acce88378"


## Op.Resolve

Retrieving key-value pairs from the FDO record for given FDO PID.

In [14]:
fdo_record_from_iri = resolve_id(iri)
fdo_record_from_handle_iri = resolve_id(handle_iri)
fdo_record_from_handle_pid = resolve_id(handle_pid)

print(fdo_record_from_iri)
print(fdo_record_from_handle_iri)
print(fdo_record_from_handle_pid)

https://raw.githubusercontent.com/knowledgepixels/nanodash/refs/heads/master/README.md
FDO Record
  ID: test-fdo-profile
  Label: Test FDO: README of Nanodash
  Profile: https://w3id.org/np/RABPR2eJ7dbuf_OPDLztvRZI-el2_wBFkVBiPCLmr1Q50/test-fdo-profile
FDO Record
  ID: 996c38676da9ee56f8ab
  Label: No label
  Profile: https://hdl.handle.net/21.T11966/996c38676da9ee56f8ab
FDO Record
  ID: 996c38676da9ee56f8ab
  Label: No label
  Profile: https://hdl.handle.net/21.T11966/996c38676da9ee56f8ab


## Op.RetrieveContent
Download the bitstream/file of an FDO.

In [15]:
from typing import Union, List, Tuple

## Utility function to print the results of this operation
def print_file_info(
    file_data: Union[
        bytes,
        Tuple[bytes, str, str],
        List[Union[bytes, Tuple[bytes, str, str]]]
    ]
):
    # Normalize to list
    if isinstance(file_data, (bytes, tuple)):
        files = [file_data]
    else:
        files = file_data

    for i, entry in enumerate(files, start=1):
        # Normalize entry
        if isinstance(entry, bytes):
            file_content, content_type, filename = entry, "unknown", None
        elif isinstance(entry, tuple):
            if len(entry) == 3:
                file_content, content_type, filename = entry
            elif len(entry) == 1:
                file_content = entry[0]
                content_type, filename = "unknown", None
            else:
                raise ValueError(f"Invalid file entry at index {i}: {entry}")
        else:
            raise ValueError(f"Unsupported file data type at index {i}: {type(entry)}")

        # Print info
        print(f"\n=== File {i} ===")
        print(f"Filename: {filename or 'N/A'}")
        print(f"Content-Type: {content_type}")
        print(f"Size: {len(file_content)} bytes")
        print("Content preview (first 500 bytes):")
        try:
            print(file_content[:500].decode("utf-8"))
        except UnicodeDecodeError:
            print(file_content[:500])  # raw bytes

# Example with FdoNanopub IRI
fdo_content_from_iri = retrieve_content_from_id(iri)
print_file_info(fdo_content_from_iri)

# Example with Handle Fdo
handle_pid_with_dataref = '21.T11975/7fcfec59-f27e-45a7-a9d1-8c9e0c64b064'
fdo_content_from_handle_pid = retrieve_content_from_id(handle_pid_with_dataref)
print_file_info(fdo_content_from_handle_pid)

# Example with multiple datarefs (Handle)
handle_pid_with_datarefs = '21.T11975/1686b432-dd80-4ec8-b8d7-be7fe1d06318'
fdo_multiple_datarefs = retrieve_content_from_id(handle_pid_with_datarefs)
print_file_info(fdo_multiple_datarefs)

https://raw.githubusercontent.com/knowledgepixels/nanodash/refs/heads/master/README.md
Content URL(s): https://raw.githubusercontent.com/knowledgepixels/nanodash/refs/heads/master/README.md

=== File 1 ===
Filename: N/A
Content-Type: unknown
Size: 2419 bytes
Content preview (first 500 bytes):
Nanodash

![logo](nanodash.png)

Nanodash was previously called Nanobench.

Nanodash is a client to browse and publish nanopublications.


### Online Instances

You can use Nanodash by login in via ORCID in one of the online instances:

- https://nanodash.petapico.org/
- https://nanodash.knowledgepixels.com/
- https://nanodash.np.trustyuri.net/


### Local Installation

To use Nanodash locally, see the [installation instructions with Docker](INSTALL-with-Docker.md).


### Screenshot

Th
https://hdl.handle.net/21.T11975/3bb724ae-32cc-4f0c-9edf-22c26859535c
Content URL(s): https://hdl.handle.net/21.T11975/3bb724ae-32cc-4f0c-9edf-22c26859535c

=== File 1 ===
Filename: N/A
Content-Type: unknown
Size: 

## Op.Validate
Validate given FDO record according to its profile.

In [16]:
# Step 1: Resolve PID
fdo_record_from_iri = resolve_id(iri)

# Step 2: Validate
validation_results_1 = validate_fdo_record(fdo_record_from_iri)
print(validation_results_1)

# TODO: examples with valid handle-based FDOs
# Step 1: Resolve PID
fdo_record_from_handle_iri = resolve_id(handle_iri)

# Step 2: Validate
validation_results_2 = validate_fdo_record(fdo_record_from_handle_iri)
print(validation_results_2)

# Step 1: Resolve PID
fdo_record_from_handle_pid = resolve_id(handle_pid)

# Step 2: Validate
validation_results_3 = validate_fdo_record(fdo_record_from_handle_pid)
print(validation_results_3)

https://raw.githubusercontent.com/knowledgepixels/nanodash/refs/heads/master/README.md
ValidationResult(is_valid=True, errors=[], warnings=[])
ValidationResult(is_valid=True, errors=[], warnings=[])
ValidationResult(is_valid=True, errors=[], warnings=[])


## Op.Create
Defining a new FDO by creating and publishing a new FDO record (for an existing file, or without file reference).

In [17]:
record = FdoRecord(
    profile_uri="https://hdl.handle.net/12345",
    label="Test FDO",
    dataref="https://example.org/data/456"
)

fdo_from_iri = FdoNanopub.create_with_fdo_iri(fdo_record=record, fdo_iri="https://example.org/example-fdo")
print(fdo_from_iri)

https://example.org/data/456
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://w3id.org/fdof/ontology#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://purl.org/nanopub/temp/np/Head> {
    <http://purl.org/nanopub/temp/np/> a np:Nanopublication ;
        np:hasAssertion <http://purl.org/nanopub/temp/np/assertion> ;
        np:hasProvenance <http://purl.org/nanopub/temp/np/provenance> ;
        np:hasPublicationInfo <http://purl.org/nanopub/temp/np/pubinfo> .
}

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> rdfs:label "FAIR Digital Object: Test FDO" ;
        npx:introduces <https://example.org/example-fdo> .
}

<http://purl.org/nanopub/temp/np/assertion> {
    <https://example.org/example-fdo> a ns1:FAIRDigitalObject ;
        rdfs:label "Test FDO" ;
        dcterms:conformsTo <https://hdl.handle.net/12345> ;
        ns1:

## Op.Handle2Nanopub

This operation is transforming an existing Handle-based FDO record into a nanopublication-based one. 

In [18]:
np = FdoNanopub.handle_to_nanopub(handle_pid)

print(np)

@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://hdl.handle.net/0.TYPE/> .
@prefix ns2: <https://hdl.handle.net/21.T11966/> .
@prefix ns3: <https://w3id.org/fdof/ontology#> .
@prefix ns4: <https://hdl.handle.net/10320/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://purl.org/nanopub/temp/np/Head> {
    <http://purl.org/nanopub/temp/np/> a np:Nanopublication ;
        np:hasAssertion <http://purl.org/nanopub/temp/np/assertion> ;
        np:hasProvenance <http://purl.org/nanopub/temp/np/provenance> ;
        np:hasPublicationInfo <http://purl.org/nanopub/temp/np/pubinfo> .
}

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> rdfs:label "FAIR Digital Object: 21.T11966/82045bd97a0acce88378" ;
        npx:introduces ns2:82045bd97a0acce88378 .
}

<http://purl.org/nanopub/temp/np/assertion> {
    ns2:82045bd97a0acce88378 a ns3:FAI

## Op.Update record

Updates the record of an FDO, keeping the same FDO PID.

In [19]:
# The following steps are not needed if starting from an existing nanopub, this is so that we can see a successful update (same pubkey)
# Create a new FDO record manually
initial_record = FdoRecord(
    profile_uri="https://w3id.org/fdo/profile/fdp", 
    label="Example FDO",
    dataref="https://example.org/dummy-data"
)

# Define a new FDO IRI 
fdo_iri = "https://example.org/test-fdo-123"

# Publish initial nanopub
npub = FdoNanopub.create_with_fdo_iri(
    fdo_record=initial_record,
    fdo_iri=fdo_iri,
    data_ref=initial_record.get_data_ref(),
    conf=conf
)

published_uri, server_used = npub.publish()

fetchConf = NanopubConf(
    use_test_server=True
)
fetchNp = Nanopub(published_uri, conf=fetchConf)
print(fetchNp)

# Step 1: Resolve existing record: source_uri will only be available if we find it in the nanopub network, 
# and it is used to update (supersede and re-publish) an existing nanopub, otherwise we will create a brand new nanopub
record = resolve_id(published_uri, conf=conf)

# Step 2: Make modifications
record.set_property(RDFS.label, Literal("Updated label"))

# Step 3: Update and publish
new_published_uri, server_used  = update_record(
    fdo_iri=published_uri,
    record=record,
    publish=True,
    conf=conf
)

print(f"FDO updated and published at: {new_published_uri} (server: {server_used})")

updatedNp = Nanopub(new_published_uri, conf=fetchConf)

print(updatedNp)


https://example.org/dummy-data
Nanopub URI: https://w3id.org/np/RAkym-ODTdchC8A1aehAuYmaSFblWQJ4epIL0uGZgKye4
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <https://w3id.org/np/RAkym-ODTdchC8A1aehAuYmaSFblWQJ4epIL0uGZgKye4/> .
@prefix this: <https://w3id.org/np/RAkym-ODTdchC8A1aehAuYmaSFblWQJ4epIL0uGZgKye4> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0009-0009-0118-9195 .
}

sub:assertion {
    <https://example.org/test-fdo-123> a ns1:FAIRDigitalObject ;
        rdfs:label "Example FDO" ;
        dcterms:conformsTo <https://w3id.org/fdo/profile/fdp> ;
        ns1:hasMetadata this: ;
        ns1:isMaterializedBy <https://exampl

## Op.Create aggregation FDO

This operation is a variant of Op.Create.

Defining a new FDO by aggregating existing FDOs.



In [20]:
fdo_iri = "https://w3id.org/fdo/example/fdo"
profile_uri= "https://w3id.org/fdo/profile/fdp"

label = "ComplexNanopubExample001"

aggregates = [
    "https://w3id.org/np/RAbb0pvoFGiNwcY8nL-qSR93O4AAcfsQRS_TNvLqt0VHg/FdoExample",
    "https://w3id.org/np/RAwCj8sM9FkB8Wyz3-i0Fh9Dcq1NniH1sErJBVEkoRQ-o/FdoExample",
    "https://w3id.org/np/RADTajQ3RJ8RNklhV8_W7B0pcJswCmm25zJPp7M-K0BRg/FdoExample",
]

npub = FdoNanopub.create_aggregation_fdo(
    fdo_iri=fdo_iri,
    profile_uri=profile_uri,
    label=label,
    aggregates=aggregates,
    conf=conf
)


npub.sign()

print(npub)
npub.publish()


Nanopub URI: https://w3id.org/np/RAl37gIr5mHkS0IBjeQ5UF01RLFQQ2l8H9479q8StTEWg
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <https://w3id.org/np/RAl37gIr5mHkS0IBjeQ5UF01RLFQQ2l8H9479q8StTEWg/> .
@prefix this: <https://w3id.org/np/RAl37gIr5mHkS0IBjeQ5UF01RLFQQ2l8H9479q8StTEWg> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0009-0009-0118-9195 .
}

sub:pubinfo {
    sub:sig npx:hasAlgorithm "RSA" ;
        npx:hasPublicKey "MIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAw3QafWPHCDnuVY+aTaavfIP9Q6GWzlYL0kTzq7+t11EKQoG6IDcxt1naJ2aftVjMmEn+n1kruckVXGXaBZANNPUJj/Vk1RGF9UqdEb+HI2F8NKxE3/lnwFWDCI4mPOI0AL5MyTOsbkzARbLeW4eMYALYuYHyjkB41OjFqh

('https://w3id.org/np/RAl37gIr5mHkS0IBjeQ5UF01RLFQQ2l8H9479q8StTEWg',
 'https://test.registry.knowledgepixels.com/np/')

## Op.Create derived FDO
This operation is a variant of Op.Create.

Defining a new FDO by deriving from existing FDOs.

In [21]:
fdo_iri = "https://w3id.org/fdo/example/fdo"
profile_uri= "https://w3id.org/fdo/profile/fdp"

label = "ComplexNanopubExample001"

sources = [
    "https://example.org/source1",
    "https://example.org/source2",
]

npub = FdoNanopub.create_derivation_fdo(
    fdo_iri=fdo_iri,
    profile_uri=profile_uri,
    label=label,
    sources=sources,
    conf=conf
)


npub.sign()

print(npub)
npub.publish()


Nanopub URI: https://w3id.org/np/RAW1-xuITkd1YO5oG_x3hv90WaYErJAOGbWSYTVa9NQAc
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix ns1: <https://w3id.org/fdof/ontology#> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sub: <https://w3id.org/np/RAW1-xuITkd1YO5oG_x3hv90WaYErJAOGbWSYTVa9NQAc/> .
@prefix this: <https://w3id.org/np/RAW1-xuITkd1YO5oG_x3hv90WaYErJAOGbWSYTVa9NQAc> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sub:provenance {
    sub:assertion prov:wasAttributedTo orcid:0009-0009-0118-9195 .
}

sub:Head {
    this: a np:Nanopublication ;
        np:hasAssertion sub:assertion ;
        np:hasProvenance sub:provenance ;
        np:hasPublicationInfo sub:pubinfo .
}

sub:assertion {
    <https://w3id.org/fdo/example/fdo> a ns1:FAIRDigitalObject ;
        rdfs:label "ComplexNanopu

('https://w3id.org/np/RAW1-xuITkd1YO5oG_x3hv90WaYErJAOGbWSYTVa9NQAc',
 'https://test.registry.knowledgepixels.com/np/')

## Op.Tombstone

To be FAIR, FDOs cannot be fully deleted. Their PID needs to remain available and allow users and applications to find out what the object was about, even when the actual content is no longer available. FDOs can for this reason be "tombstoned", i.e. marked as obsolete and removed, but with a visible sign of its previous existence.

In [22]:
retraction = NanopubRetract(conf, npub.source_uri)
print(retraction)

@prefix np: <http://www.nanopub.org/nschema#> .
@prefix npx: <http://purl.org/nanopub/x/> .
@prefix orcid: <https://orcid.org/> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://purl.org/nanopub/temp/np/Head> {
    <http://purl.org/nanopub/temp/np/> a np:Nanopublication ;
        np:hasAssertion <http://purl.org/nanopub/temp/np/assertion> ;
        np:hasProvenance <http://purl.org/nanopub/temp/np/provenance> ;
        np:hasPublicationInfo <http://purl.org/nanopub/temp/np/pubinfo> .
}

<http://purl.org/nanopub/temp/np/pubinfo> {
    <http://purl.org/nanopub/temp/np/> prov:generatedAtTime "2025-07-11T08:06:09.135741"^^xsd:dateTime ;
        prov:wasAttributedTo orcid:0009-0009-0118-9195 .
}

<http://purl.org/nanopub/temp/np/provenance> {
    <http://purl.org/nanopub/temp/np/assertion> prov:generatedAtTime "2025-07-11T08:06:09.135741"^^xsd:dateTime ;
        prov:wasAttributedTo orcid:0009-0009-0118-9195 .
}

<http://purl.org/nano